# Sentiment Analysis

In [1]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("LondonStory/txlm-roberta-hindi-sentiment")
sentimen_analysis_model = AutoModelForSequenceClassification.from_pretrained("LondonStory/txlm-roberta-hindi-sentiment").to(device)

B:\Software\Anaconda\envs\predixion\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detai

In [17]:
input = 'हाँ-हाँ, मैं समझ गया। मैं पूरी कोशिश करूँगा।'
encoded_input = tokenizer(input, return_tensors="pt").to(device)

In [19]:
encoded_input

{'input_ids': tensor([[     0, 131952,      9, 150858,      4,  10399,  41126,   5349,    125,
          10399,  41060,  77711,  33675,   4739,  10307,    125,      2]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [14]:
with torch.no_grad():
    output = sentimen_analysis_model(**encoded_input).logits

In [15]:
predicted_class_id = output.argmax().item()
sentimen_analysis_model.config.id2label[predicted_class_id]

'LABEL_2'

Label 1- neg, Label 0- neutral, Label-2 pos

In [24]:
sentimen_analysis_model.config

XLMRobertaConfig {
  "_name_or_path": "LondonStory/txlm-roberta-hindi-sentiment",
  "architectures": [
    "XLMRobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2
  },
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "xlm-roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "transformers_version": "4.32.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 250002
}

In [7]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="LondonStory/txlm-roberta-hindi-sentiment")

In [8]:
pipe('नमस्ते, मैं राजेश बोल रहा हूँ, CFI फाइनेंस कंपनी से। क्या मैं श्री सुरेश कुमार से बात कर सकता हूँ?')

[{'label': 'LABEL_2', 'score': 0.516538679599762}]

# Summarizer

In [ ]:
convo = '''Speaker 1: नमस्ते, मैं राजेश बोल रहा हूँ, CFI फाइनेंस कंपनी से। क्या मैं श्री सुरेश कुमार से बात कर सकता हूँ?
Speaker 2: हाँ, मैं ही सुरेश बोल रहा हूँ। क्या बात है?
Speaker 1: सुरेशजी, मैं आपके लोन के बारे में बात करने के लिए फोन कर रहा हूँ। आपका EMI पिछले तीन महीने से बकाया है।
Speaker 2: अरे हाँ, मुझे पता है। लेकिन आप लोग हर रोज फोन करके परेशान कर रहे हैं। मैंने कहा था ना कि जैसे ही पैसे होंगे, मैं भर दूंगा।
Speaker 1: देखिए सुरेशजी, हम समझते हैं आपकी परेशानी। लेकिन आप भी समझिए, यह हमारा काम है। आपको पता है कि आपके ऊपर कितना बकाया है?
Speaker 2: हाँ-हाँ, मुझे अंदाजा है। लगभग 50,000 रुपये होंगे।
Speaker 1: नहीं सुरेशजी, आप गलत हैं। आप पर कुल 75,000 रुपये बकाया है, जिसमें लेट फीस और ब्याज भी शामिल है।
Speaker 2: क्या? इतने ज्यादा? यह कैसे हो सकता है? आप लोग लूट रहे हैं क्या?
Speaker 1: देखिए सुरेशजी, गुस्सा करने से कुछ नहीं होगा। आपको समय पर पैसे जमा करने चाहिए थे। अब बताइए, कब तक आप यह राशि जमा कर सकते हैं?
Speaker 2: अरे भाई, मेरी बात तो सुनो! मेरी माँ की तबीयत खराब है, अस्पताल में भर्ती हैं। सारे पैसे उनके इलाज में खर्च हो गए।
Speaker 1:  सुरेशजी, मुझे आपकी परिस्थिति से कोई मतलब नहीं है। हमें बस अपना पैसा चाहिए। आप समझ रहे हैं या नहीं?
Speaker 2: क्या बकवास कर रहे हो? तुम्हें शर्म नहीं आती? मेरी माँ मरने की कगार पर हैं और तुम पैसों की बात कर रहे हो?
Speaker 1:  देखो सुरेश, मुझे तुम्हारी कहानियों से कोई फर्क नहीं पड़ता। तुमने लोन लिया है तो चुकाना पड़ेगा। समझे? वरना हम कानूनी कार्रवाई करेंगे।
Speaker 2: तुम धमकी दे रहे हो? तुम्हें पता है मैं कौन हूँ? मैं तुम्हारी नौकरी गंवा सकता हूँ।
Speaker 1:  अरे वाह! अब तुम भी धमकी दे रहे हो? चलो, देखते हैं कौन क्या कर सकता है। तुम अपनी माँ का इलाज करवाओ, मैं तुम्हारा इलाज करूँगा।
Speaker 2: बदतमीज! तुम्हारी हिम्मत कैसे हुई ऐसे बात करने की? मैं तुम्हारे बॉस से शिकायत करूँगा।
Speaker 1: करो जो करना है। लेकिन याद रखना, अगर कल तक पैसे नहीं आए तो मैं खुद तुम्हारे घर आऊंगा। और तब देखेंगे कौन किसका इलाज करता है।
Speaker 2: तुम मेरे घर आओगे? ठीक है, आ जाओ। देखता हूँ कैसे आते हो। मैं पुलिस को बुला लूंगा।
Speaker 1: अरे वाह! पुलिस? बहुत अच्छा। हम भी देखेंगे कि पुलिस किसका साथ देती है - एक कर्जदार का या एक कानूनी कंपनी का।
Speaker 2: तुम... तुम बहुत बुरे इंसान हो। मेरी मजबूरी का फायदा उठा रहे हो।
Speaker 1:  हाँ-हाँ, मैं बहुत बुरा हूँ। लेकिन तुमसे तो अच्छा ही हूँ। कम से कम मैं अपने वादे तो पूरे करता हूँ।
Speaker 2: क्या मतलब? मैंने कभी वादा तोड़ा है क्या?
Speaker 1: अरे सुरेश भाई, तुमने हर महीने EMI भरने का वादा किया था। वो क्या हुआ? भूल गए?
Speaker 2:  देखो, मैं समझता हूँ। लेकिन अभी मेरे पास सच में पैसे नहीं हैं। क्या कोई और तरीका नहीं है?
Speaker 1: तरीका तो है। लेकिन उसके लिए तुम्हें कुछ करना होगा।
Speaker 2: क्या करना होगा? बताओ, मैं तैयार हूँ।
Speaker 1: पहले तो अपनी माँ का इलाज बंद करो। उन पैसों से हमारा कर्ज चुका दो।
Speaker 2:  क्या बकवास कर रहे हो? तुम पागल हो गए हो क्या?
Speaker 1: अरे नहीं-नहीं, मैं तो मजाक कर रहा था। तुम तो बहुत जल्दी गुस्सा हो जाते हो।
Speaker 2:  तुम्हें यह सब मजाक लगता है? मेरी जिंदगी बर्बाद हो रही है और तुम मजाक कर रहे हो?
Speaker 1: अच्छा सुनो, मैं तुम्हें एक और मौका देता हूँ। कल तक 25,000 रुपये जमा कर दो। बाकी के लिए हम एक नया प्लान बना लेंगे।
Speaker 2: 25,000? इतने पैसे कहाँ से लाऊँ मैं?
Speaker 1: वो तुम्हारी समस्या है। चाहे दोस्तों से उधार लो, या फिर कुछ बेच दो। लेकिन कल तक पैसे चाहिए।
Speaker 2: तुम समझ क्यों नहीं रहे हो? मेरे पास सच में पैसे नहीं हैं।
Speaker 1: देखो सुरेश, मुझे लगता है तुम समझ नहीं रहे हो। अगर तुम कल तक पैसे नहीं देते, तो मैं तुम्हारे ऑफिस आऊंगा और सबके सामने तुम्हारी पोल खोल दूंगा।
Speaker 2:  नहीं-नहीं, ऐसा मत करना। मेरी इज्जत का सवाल है।
Speaker 1: तो फिर ठीक है। कल तक पैसे जमा कर दो। वरना तुम जानते हो क्या होगा।
Speaker 2: ठीक है, मैं कोशिश करूँगा। लेकिन प्लीज, मेरे ऑफिस मत आना।
Speaker 1: बहुत अच्छा। अब तुम समझदारी की बात कर रहे हो। याद रखना, कल शाम 5 बजे तक पैसे नहीं आए तो मैं सीधा तुम्हारे ऑफिस पहुँच जाऊंगा।
Speaker 2: हाँ-हाँ, मैं समझ गया। मैं पूरी कोशिश करूँगा।
Speaker 1: ठीक है। और हाँ, अगली बार जब लोन लो तो याद रखना - समय पर पैसे नहीं चुकाओगे तो ऐसे ही परेशान होना पड़ेगा।
Speaker 2:  हाँ, मैं समझ गया। अब मैं रखता हूँ।
Speaker 1: ठीक है। और याद रखना, कल शाम 5 बजे। एक मिनट की भी देरी हुई तो...
Speaker 2: हाँ-हाँ, मैंने कहा ना मैं कोशिश करूँगा।
Speaker 1: कोशिश नहीं, करना होगा। समझे? अच्छा, अब रखता हूँ। कल मिलते हैं - या तो बैंक में, या फिर तुम्हारे ऑफिस में।
Speaker 2:  हाँ, ठीक है। नमस्ते।
'''

In [19]:
import googletrans

In [37]:
with open('convo.txt', "r", encoding="utf-8") as f:
    hindi_text = f.read()

hindi_text = 'नमस्ते, मैं राजेश बोल रहा हूँ, CFI फाइनेंस कंपनी से। क्या मैं श्री सुरेश कुमार से बात कर सकता हूँ?'

In [39]:
try:
    translator = googletrans.Translator()
    english_text = translator.translate(hindi_text, dest="en").text
except Exception as e:
    print("Translation failed:", e)
    english_text = ""

In [41]:
english_text

'Hello, I am Rajesh from CFI Finance Company. Can I speak to Mr. Suresh Kumar?'

In [3]:
from transformers import AutoTokenizer
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

checkpoint = "Jayveersinh-Raj/hindi-summarizer-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
summarizer_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint).to("cuda" if torch.cuda.is_available() else "cpu")


B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
B:\Software\Anaconda\envs\predixion\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more detai

In [71]:
import re
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

model_name = "csebuetnlp/mT5_m2o_hindi_crossSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
sum_model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

B:\Software\Anaconda\envs\predixion\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. If you see this, DO NOT PANIC! This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=True`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\convert_slow_tokenizer.py:470: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback opt

In [74]:
article_text = 'सुरेशजी, मैं आपके लोन के बारे में बात करने के लिए फोन कर रहा हूँ। आपका EMI पिछले तीन महीने से बकाया है।'

input_ids = tokenizer(
    [WHITESPACE_HANDLER(article_text)],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

output_ids = sum_model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

In [76]:
output_ids

tensor([250021,    259,  21726,  22753,  15230,  32954,   9255,  10596,    641,
         26980,   1462, 116277,  88306,  36086,  20976,    259,  30590,   1163,
          1890,    259,    863,   2075,   1822,  36158,  19422,    757,    943,
          2075,    830,    260,      1])

In [81]:
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

In [83]:
summary

'भारतीय रिज़र्व बैंक के पूर्व गवर्नर सुरेश कुमार का एक नया बयान सामने आया है.'

# NER

In [3]:
from transformers import AutoModelForTokenClassification, AutoTokenizer
import torch

ner_model = AutoModelForTokenClassification.from_pretrained("MichaelHuang/muril_base_cased_hindi_ner")
tokenizer = AutoTokenizer.from_pretrained("google/muril-base-cased")

# Define the labels dictionary
labels_dict = {
    0: "B-FESTIVAL",
    1: "B-GAME",
    2: "B-LANGUAGE",
    3: "B-LITERATURE",
    4: "B-LOCATION",
    5: "B-MISC",
    6: "B-NUMEX",
    7: "B-ORGANIZATION",
    8: "B-PERSON",
    9: "B-RELIGION",
    10: "B-TIMEX",
    11: "I-FESTIVAL",
    12: "I-GAME",
    13: "I-LANGUAGE",
    14: "I-LITERATURE",
    15: "I-LOCATION",
    16: "I-MISC",
    17: "I-NUMEX",
    18: "I-ORGANIZATION",
    19: "I-PERSON",
    20: "I-RELIGION",
    21: "I-TIMEX",
    22: "O"
}

B:\Software\Anaconda\envs\predixion\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\utils\generic.py:260: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
B:\Software\Anaconda\envs\predixion\lib\site-packages\huggingface_hub\file_download.py:1150: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/948M [00:00<?, ?B/s]

B:\Software\Anaconda\envs\predixion\lib\site-packages\transformers\modeling_utils.py:479: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(checkpoint_file, ma

tokenizer_config.json:   0%|          | 0.00/206 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/3.16M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/113 [00:00<?, ?B/s]

In [5]:
sentence = "नमस्ते, मैं राजेश बोल रहा हूँ, CFI फाइनेंस कंपनी से। क्या मैं श्री सुरेश कुमार से बात कर सकता हूँ? हाँ, मैं ही सुरेश बोल रहा हूँ। क्या बात है?"
inputs = tokenizer(sentence, return_tensors="pt", padding=True, truncation=True, max_length=128)

In [9]:
with torch.no_grad():
  output = ner_model(**inputs).logits

In [11]:
predicted_labels = output.argmax(-1)


labels = predicted_labels.squeeze().tolist()
predicted_labels = [labels_dict[label] for label in labels]

In [13]:
tokens = tokenizer.convert_ids_to_tokens(inputs["input_ids"][0])

result = list(zip(tokens, predicted_labels))

for token, label in result:
  if label != "O":
    print(f"{token}: {label}")

राजेश: B-PERSON
CF: B-ORGANIZATION
##I: B-ORGANIZATION
फाइनेंस: I-ORGANIZATION
कंपनी: I-ORGANIZATION
श्री: B-PERSON
सुरेश: B-PERSON
कुमार: I-PERSON
सुरेश: B-PERSON
